In [ ]:
#@title Paste token below and press ▶️  button to the left of this title { display-mode: "form" }
# %tb

# config={
#     'authorizationToken':'',
#     'dataset_id': "solucio-vgc-member-survey-working_sentences",
#     "vector_field": "q06_sentences_mpnet_vector_",
#     "text_field" : 'Q06_sentences',
#     "output_field": "kmeans-new_cluster_",
#     "min_n_clusters": 50,
#     "max_n_clusters": 60,
#     "method": 'kmeans', # select from 'dbscan' or 'kmeans' or 'hybrid'
# }


In [ ]:
token = "" #@param {type:"string"}

In [ ]:
# print("Installing ...")
# !pip install clusteval
# !pip install -q RelevanceAI
# !pip install -U sentence-transformers
# !pip install sentence-splitter
# !pip install clusteval
# !pip install bertopic

In [ ]:
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from sentence_transformers import SentenceTransformer
from clusteval import clusteval
import numpy as np
from bertopic import BERTopic
from relevanceai import Client

In [ ]:
from relevanceai.utils import decode_workflow_token
config = decode_workflow_token(token)

authorizationToken = config['authorizationToken']
dataset_id = config['dataset_id']
vector_field = config['vector_field']
text_field = config['text_field']
alias = config['output_field']

min_n_clusters = int(config['min_n_clusters'])
max_n_clusters = int(config['max_n_clusters'])
method = config['method']

In [ ]:
client = Client(token=authorizationToken)
ds = client.Dataset(dataset_id)
docs = ds.get_all_documents(filters=ds[vector_field].exists(), select_fields=[text_field, vector_field])

sents = []
for doc in docs:
  sents.append(doc[text_field])

In [ ]:
def cluster_topic(texts, ids, labels, n_gram_range, diversity, decay, thres):
  # if len(ids) < thres:
  #   return texts, ids, labels
  try:
    topic_model = BERTopic(n_gram_range=n_gram_range,diversity=diversity*(1+decay), min_topic_size=9)
    topics, probs = topic_model.fit_transform(texts)
  except:
    return texts, ids, labels
#   print('topic_info:')
#   print(topic_model.get_topic_info())
  n_topics = len(set(topics))
  if n_topics == 1:
    return texts, ids, labels
  clusters = {}
  for topic_id in set(topics):
    clusters[topic_id] = []
  for idx, topic_id in enumerate(topics):
    new_label = str(labels[idx]) + '_' + str(topic_id)
    clusters[topic_id].append((texts[idx], ids[idx], new_label))
  all_texts = []
  all_ids = []
  all_labels = []
  for topic_id in set(topics):
    cur_texts = [info[0] for info in clusters[topic_id]]
    cur_ids = [info[1] for info in clusters[topic_id]]
    cur_labels = [info[2] for info in clusters[topic_id]]
    print(str(cur_labels[0]) + ": " + str(len(clusters[topic_id])))
    if len(clusters[topic_id]) < thres:
      print("pass")
      new_texts = cur_texts
      new_ids = cur_ids
      new_labels = cur_labels
    else:
      print("split")
      new_texts, new_ids, new_labels = cluster_topic(cur_texts, cur_ids, cur_labels, n_gram_range, diversity, decay, thres)
    all_texts = all_texts + new_texts
    all_ids = all_ids + new_ids
    all_labels = all_labels + new_labels
  return all_texts, all_ids, all_labels

In [ ]:
class One_to_many_cluster:
  def __init__(self, sents, config) -> None:
    self.sents = sents
    # self.ids = []
    # for i, doc in enumerate(docs):
    #   sents.append(doc[new_text_field])
    #   ids.append(i)
    self.method = config['method']
    self.min_n_clusters = int(config["min_n_clusters"])
    self.max_n_clusters = int(config["max_n_clusters"])
  
  def fit_predict(self, data):
    if self.method == 'kmeans':
      # model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
      # vectors = model.encode(self.sents)
      ce = clusteval(cluster='kmeans', evaluate='silhouette', min_clust=self.min_n_clusters, max_clust=self.max_n_clusters,metric='cosine')
      result = ce.fit(np.array(data))
      # print('ha5')
      labels = result['labx']
      return labels
    elif self.method == 'dbscan':
      ids = []
      labels = []
      for i, t in enumerate(self.sents):
        ids.append(i)
        labels.append("")

      res_texts, res_ids, res_labels = cluster_topic(self.sents, ids, labels, n_gram_range=(1,3), diversity=0, decay=0, thres=10)
      so = np.argsort(res_ids)
      labels = np.array(res_labels)[so]
      ids = np.array(res_ids)[so]
      return labels
    elif self.method == 'hybrid':
      ce = clusteval(cluster='kmeans', evaluate='silhouette', min_clust=self.min_n_clusters, max_clust=self.max_n_clusters,metric='cosine')
      result = ce.fit(np.array(data))
      labels = result['labx']

      res_texts = []
      res_ids = []
      res_labels = []
      for label in set(labels):
        cur_labels = []
        cur_texts = []
        cur_ids = []
        for i, l in enumerate(labels):
          if label == l:
            cur_labels.append(l)
            cur_ids.append(i)
            cur_texts.append(self.sents[i])
#         print('start bertopic: {}'.format(len(cur_ids)))
#         print('==========')
        new_texts, new_ids, new_labels = cluster_topic(cur_texts, cur_ids, cur_labels, n_gram_range=(1,3), diversity=0, decay=0, thres=10)
        res_texts = res_texts + new_texts
        res_ids = res_ids + new_ids
        res_labels = res_labels + new_labels
      so = np.argsort(res_ids)
      labels = np.array(res_labels)[so]
      ids = np.array(res_ids)[so]
      return labels

In [ ]:
model = One_to_many_cluster(sents, config)
ds.cluster(vector_fields = [vector_field], model = model, alias = alias)

ds.update_field_children(
    field=vector_fields[0],
    field_children=[".".join(["_cluster_", vector_fields[0], alias])],
    category="cluster"
)

workflow_id = job_id
client.workflows.status(
    workflow_id=workflow_id,
    status="complete",
    workflow_name="Auto Cluster",
    metadata={},
    additional_information="We have clustered your data."
)